# Product-mix Pyomo

$$
\begin{align}
    \text{max} \quad & \sum_{j \in J} c_j x_j \\
    \text{s.t.} \quad & \sum_{j \in J} a_{i, j} x_{j} \leq b_{i} & \forall \; i \in I \\
    & x_{j} \geq 0 & \forall \; j \in J \\
\end{align}
$$

In [23]:
# Python native modules
import json

# Third-party packages
import pyomo.environ as pyo

In [24]:
# Read input file and store in local variable `input_data`
with open("input_prod_mix.json", mode="r", encoding="utf8") as file:
    input_data = json.load(file)

In [25]:
availabilities = {b["resource"]: b["value"] for b in input_data["availabilities"]}
margins = {c["product"]: c["value"] for c in input_data["margins"]}
proportions = {
    (p["resource"], p["product"]): p["proportion"]
    for p in input_data["proportions"]
}

In [29]:
# Create model instance
model = pyo.ConcreteModel()

In [30]:
# Sets for resources I and products J
model.I = pyo.Set(initialize=availabilities.keys())
model.J = pyo.Set(initialize=margins.keys())

In [31]:
# Paramters
model.c = pyo.Param(model.J, initialize=margins)
model.b = pyo.Param(model.I, initialize=availabilities)
model.a = pyo.Param(model.I, model.J, initialize=proportions)

In [32]:
# Decision variables
model.x = pyo.Var(model.J, within=pyo.NonNegativeReals)

In [33]:
# Resource availablity constraints

# ! constrainted applied on resources, saying that :
# ! for every resource (i),
# ! the quantity of that resource used in the production of all (that's why we are summing over j) products
# ! should be less than or equal to the availability of that resource (indexed by i -> b[i])

def av_cstr(model, i):
    return sum(model.a[i,j] * model.x[j] for j in model.J) <= model.b[i]

# ! the av_cstr rule `rule=av_cstr` is applied over all resources in the set I `model.I`

model.av_cstr = pyo.Constraint(model.I, rule=av_cstr)

In [34]:
# Objective function
def obj(model):
    return sum(model.c[j] * model.x[j] for j in model.J)

model.obj = pyo.Objective(rule=obj, sense=pyo.maximize)

In [35]:
# Instantiate Highs persistent solver (make sure highspy is installed)
solver = pyo.SolverFactory("appsi_highs")

In [36]:
# Apply method solve
solver.solve(model)

{'Problem': [{'Lower bound': 258.8526315789474, 'Upper bound': 258.8526315789474, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [37]:
# Use objective as a callable to see its value
model.obj()

258.8526315789474

In [38]:
# Print results
for j, xi in model.x.extract_values().items():
    print(f"{xi:.2f} units of Product {j}")

71.05 units of Product P1
10.26 units of Product P2
53.68 units of Product P3


In [67]:
for x in model.x.values():
    print(f'variable {x.name} with value : {x.lb} <= {x.value:.2f} <= {x.ub}')
    # print(x.value)
    # print(x.name)  # print(x.getname())
    # print(x.lb)    # print(x.lower)
    # print(x.ub)    # print(x.upper)

variable x[P1] with value : 0 <= 71.05 <= None
variable x[P2] with value : 0 <= 10.26 <= None
variable x[P3] with value : 0 <= 53.68 <= None
